In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import pickle
from os.path import abspath

In [2]:
#add seeds for reproducability
import numpy as np
import random
np.random.seed(42)
random.seed(42)

In [3]:
stimuli_path = abspath('../../scrambled_stimuli_dfs/')
print(stimuli_path)

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/ressources/scrambled_stimuli_dfs


In [4]:
with open(os.path.join(stimuli_path, "stimuli_Original.pkl"), "rb") as f:
    stimuli_df = pickle.load(f)
stimuli_df.head()

Loading lookup from /om2/user/ckauf/anaconda/envs/perturbedenv/lib/python3.6/site-packages/brainio_collection/lookup.csv
/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/neural_nlp/../ressources/stimuli


 We're running in the NEW version of the implementations.py script.




/om2/user/ckauf/anaconda/envs/perturbedenv/lib/python3.6/site-packages/brainio_base/assemblies.py:224: FutureWarning:

The inplace argument has been deprecated and will be removed in a future version of xarray.



,sentence,sentence_num,stimulus_id,experiment,story,passage_index,passage_label,passage_category
0,beekeeping encourages the conservation of loca...,0,243sentences.0,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
1,it is in every beekeeper's interest to conserv...,1,243sentences.1,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
2,as a passive form of agriculture it does not r...,2,243sentences.2,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
3,beekeepers also discourage the use of pesticid...,3,243sentences.3,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
4,artisanal beekeepers go to extremes for their ...,4,243sentences.4,243sentences,243sentences.beekeeping,2,beekeeping,beekeeping


In [5]:
stimulus_ids = list(stimuli_df['stimulus_id'])

conditions = ["Original", "Scr1", "Scr3", "Scr5", "Scr7", "lowPMI"]

# prepare stimuli lists for shuffling

In [6]:
def get_stimuli_dict(stimuli_path):
    stimuli_dict = {}
    for filename in os.listdir(stimuli_path):
        if ".ipynb_checkpoints" in filename:
            continue
        condition = filename.split("stimuli_")[1].split(".pkl")[0]
        print(condition)
        if condition in conditions:
            with open(os.path.join(stimuli_path,filename), "rb") as f:
                df = pickle.load(f)
                stimuli = list(df['sentence'])
                stimulus_ids = list(df['stimulus_id'])
                stimuli_dict[condition] = list(zip(stimulus_ids, stimuli))
    return stimuli_dict

In [7]:
stimuli_dict = get_stimuli_dict(stimuli_path)
stimuli_dict

Original
Scr1
Scr3
Scr5
Scr7
backward
contentwords
functionwords
lowPMI
nouns
nounsverbs
nounsverbsadj
random
random_withreplacement
randomnouns
randomnouns_withreplacement
sentenceshuffle-random
sentenceshuffle-withinpassage
verbs


{'Original': [('243sentences.0',
   'beekeeping encourages the conservation of local habitats.'),
  ('243sentences.1',
   "it is in every beekeeper's interest to conserve local plants that produce pollen."),
  ('243sentences.2',
   'as a passive form of agriculture it does not require that native vegetation be cleared to make way for crops.'),
  ('243sentences.3',
   'beekeepers also discourage the use of pesticides on crops because they could kill the honeybees.'),
  ('243sentences.4',
   'artisanal beekeepers go to extremes for their craft but their product is worth the effort.'),
  ('243sentences.5',
   'artisanal honey-making emphasizes quality and character over quantity and consistency.'),
  ('243sentences.6',
   'to produce the finest honey beekeepers become micromanagers of their honeybees.'),
  ('243sentences.7',
   'they scout the fields know when nectar flows and select the best ways to extract honey.'),
  ('243sentences.8',
   'as the beekeeper opens the hive the deep hum o

# For each sentence prepare 10 random orderings and assert that they are not in any of the other lists

In [8]:
import random

#generate 10 random permutations of each sequence
def perm_generator(seq, seen):
    np.random.seed(42)
    random.seed(42)

    seen = set(seen)
    #print(len(seen))
    length = len(seq)
    while True:
        perm = tuple(random.sample(seq, length))
        if perm not in seen:
            seen.add(perm)
            #print(len(seen))
            yield perm
        else:
            print('Already saw this permutation!')
        #print('*'*30)

In [9]:
N = 10 #number of desired shufflings
def get_10_random_perms(stimuli_dict, N):
    np.random.seed(42)
    random.seed(42)

    ten_random_permutations_dict = {}

    for ind, (identifier, elm) in enumerate(stimuli_dict['Original']):
        
        print(ind, identifier, elm)
        words = elm.split()
        words = [word.rstrip(".") for word in words]

        #add permutations of other conditions of this sentence to the "seen" set s.th. the generated
        #permutations do not include already seen shuffling conditions
        seen = [tuple(stimuli_dict[cond][ind][1].split()) for cond in conditions]
        rand_perms = perm_generator(words, seen)
        perms = [next(rand_perms) for _ in range(N)]
        perms = [list(elm) for elm in perms]
        
        #additional check for mutual exclusivity
        scr_conds = [list(elm) for elm in seen]
        for elm in scr_conds:
            assert elm not in perms
        for elm in perms:
            assert elm not in scr_conds
            
        perms = [' '.join(elm) + "." for elm in perms]
            
        print(perms)
        print('\n')
        ten_random_permutations_dict[identifier] = perms
        
    return ten_random_permutations_dict

In [10]:
ten_random_permutations_dict = get_10_random_perms(stimuli_dict, N)

0 243sentences.0 beekeeping encourages the conservation of local habitats.
['local beekeeping habitats the of conservation encourages.', 'local beekeeping of habitats the encourages conservation.', 'beekeeping habitats encourages of the local conservation.', 'local habitats of conservation beekeeping encourages the.', 'habitats beekeeping encourages conservation of the local.', 'encourages the beekeeping of habitats conservation local.', 'habitats the of local beekeeping encourages conservation.', 'conservation beekeeping of the habitats encourages local.', 'local beekeeping habitats encourages conservation of the.', 'habitats beekeeping conservation the encourages of local.']


1 243sentences.1 it is in every beekeeper's interest to conserve local plants that produce pollen.
["that is it beekeeper's every local produce interest pollen plants in to conserve.", "it is every that local pollen beekeeper's produce to plants conserve interest in.", "pollen it in to interest beekeeper's is p

['heat can band-aids risk the holding and of moisture in increase infection by.', 'band-aids can the heat holding moisture risk and infection in by of increase.', 'moisture band-aids increase infection of risk can in heat and the holding by.', 'of moisture in risk band-aids by heat holding the and increase can infection.', 'in the can band-aids and risk moisture heat infection holding of increase by.', 'increase of and the risk can holding heat by infection band-aids in moisture.', 'risk heat holding the of band-aids can by increase in infection and moisture.', 'in and of the by infection heat moisture can increase band-aids holding risk.', 'and in infection moisture heat of can holding risk the band-aids increase by.', 'increase heat moisture infection can in the risk by and of band-aids holding.']


160 243sentences.160 fortunately most skin infections can easily be treated once they are diagnosed.
['are most fortunately can infections diagnosed be they once skin easily treated.', 'f

['coffee is coffee coffee from plant the the of made beans seeds.', 'coffee is from the plant coffee coffee the of seeds beans made.', 'coffee made the beans coffee is the coffee plant from of seeds.', 'beans plant the coffee coffee coffee from of seeds is the made.', 'coffee the beans from is coffee plant seeds made the of coffee.', 'the coffee seeds beans made the plant is of from coffee coffee.', 'of from made seeds the the beans coffee is plant coffee coffee.', 'from coffee beans the coffee coffee is seeds made of the plant.', 'coffee seeds made coffee the is beans of plant from the coffee.', 'beans from made of seeds coffee the plant is the coffee coffee.']


324 384sentences.81 coffee is a popular drink in many countries and usually served hot.
['served is coffee drink popular hot many usually and a in countries.', 'coffee is popular usually hot drink served many and countries in a.', 'coffee a many in drink is usually served hot popular and countries.', 'in hot usually drink cof

489 384sentences.246 peaches have a large red-brown stone inside which contains the seed.
['seed have peaches red-brown the inside stone contains a large which.', 'peaches seed have large contains red-brown stone the a which inside.', 'large which red-brown peaches inside have seed a stone the contains.', 'stone have the inside peaches a seed red-brown which contains large.', 'inside have contains red-brown seed stone which a large peaches the.', 'peaches large red-brown have inside which seed the stone a contains.', 'a stone the large contains seed peaches have inside red-brown which.', 'which inside red-brown large stone a peaches have contains seed the.', 'red-brown have large stone the which contains seed a inside peaches.', 'red-brown a large seed stone which contains inside have peaches the.']


490 384sentences.247 peaches are sweet and delicate and must be harvested after ripening.
['ripening are peaches delicate after must and harvested sweet and be.', 'peaches ripening are an

In [11]:
print(list(ten_random_permutations_dict.keys())[:10])
print(len(ten_random_permutations_dict['243sentences.0']))
print(ten_random_permutations_dict['243sentences.0'][:10][0])

['243sentences.0', '243sentences.1', '243sentences.2', '243sentences.3', '243sentences.4', '243sentences.5', '243sentences.6', '243sentences.7', '243sentences.8', '243sentences.9']
10
local beekeeping habitats the of conservation encourages.


# Create df for our randomly permuted stimuli

In [12]:
#os.chdir(os.path.join(root_dir, 'add_random_lowPMI_condition'))
list(ten_random_permutations_dict.keys())[:10]

['243sentences.0',
 '243sentences.1',
 '243sentences.2',
 '243sentences.3',
 '243sentences.4',
 '243sentences.5',
 '243sentences.6',
 '243sentences.7',
 '243sentences.8',
 '243sentences.9']

In [13]:
def flatten_array(liste):
    liste_flatten = [item for sublist in liste for item in sublist]
    return liste_flatten

In [14]:
stimulus_ids_lst = flatten_array([[elm] * 10 for elm in list(ten_random_permutations_dict.keys())])
sentence_lst = flatten_array(list(ten_random_permutations_dict.values()))

print(len(stimulus_ids_lst), len(sentence_lst))
print(stimulus_ids_lst[:20])
print(sentence_lst[:20])

6270 6270
['243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.0', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1', '243sentences.1']
['local beekeeping habitats the of conservation encourages.', 'local beekeeping of habitats the encourages conservation.', 'beekeeping habitats encourages of the local conservation.', 'local habitats of conservation beekeeping encourages the.', 'habitats beekeeping encourages conservation of the local.', 'encourages the beekeeping of habitats conservation local.', 'habitats the of local beekeeping encourages conservation.', 'conservation beekeeping of the habitats encourages local.', 'local beekeeping habitats encourages conservation of the.', 'habitats beekeeping conservation the encourages of local.', "that is it beekee

In [15]:
def get_10_random_perms_df(ten_random_permutations_dict):
    stimulus_ids_lst = flatten_array([[elm] * 10 for elm in list(ten_random_permutations_dict.keys())])
    sentence_lst = flatten_array(list(ten_random_permutations_dict.values()))
    
    df = pd.DataFrame(
        {'stimulus_id' : stimulus_ids_lst,
        'permutated_stimulus' : sentence_lst})
    
    return df

In [16]:
random_df = get_10_random_perms_df(ten_random_permutations_dict)
random_df

,stimulus_id,permutated_stimulus
0,243sentences.0,local beekeeping habitats the of conservation ...
1,243sentences.0,local beekeeping of habitats the encourages co...
2,243sentences.0,beekeeping habitats encourages of the local co...
3,243sentences.0,local habitats of conservation beekeeping enco...
4,243sentences.0,habitats beekeeping encourages conservation of...
...,...,...
6265,384sentences.383,woman reproductive than organs a has different...
6266,384sentences.383,a different than has man woman reproductive a ...
6267,384sentences.383,a organs woman has man than reproductive diffe...
6268,384sentences.383,has different woman a organs man reproductive ...


In [17]:
test_df = random_df.loc[random_df['stimulus_id'] == '243sentences.0']
test_df

,stimulus_id,permutated_stimulus
0,243sentences.0,local beekeeping habitats the of conservation ...
1,243sentences.0,local beekeeping of habitats the encourages co...
2,243sentences.0,beekeeping habitats encourages of the local co...
3,243sentences.0,local habitats of conservation beekeeping enco...
4,243sentences.0,habitats beekeeping encourages conservation of...
5,243sentences.0,encourages the beekeeping of habitats conserva...
6,243sentences.0,habitats the of local beekeeping encourages co...
7,243sentences.0,conservation beekeeping of the habitats encour...
8,243sentences.0,local beekeeping habitats encourages conservat...
9,243sentences.0,habitats beekeeping conservation the encourage...


In [18]:
random_df.to_csv('0_into_calculatePMI_10randompermutations.csv',header=False)

In [19]:
import pickle

sentences = []
sample = []
with open('0_into_calculatePMI_10randompermutations.csv') as f:
    for line in f.readlines():
        entry = line.split(',')
        print(entry)
        sentences.append(entry)
        sample.append(entry[2])
        print('Sentence: ', sentences)
        print('Sample: ',sample)
        break

['0', '243sentences.0', 'local beekeeping habitats the of conservation encourages.\n']
Sentence:  [['0', '243sentences.0', 'local beekeeping habitats the of conservation encourages.\n']]
Sample:  ['local beekeeping habitats the of conservation encourages.\n']


# How to run afterwards

* module load openmind/miniconda/4.0.5-python3
* sbatch 1_run_grabngrams.sh
* sbatch 2_run_pPMI.sh